In [3]:
!pip3 install opencv-python

In [1]:
 import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torchvision import datasets, transforms
from sklearn.cluster import MiniBatchKMeans
import tqdm

ModuleNotFoundError: No module named 'cv2'

In [ ]:

# 加载Caltech101数据集
transform = transforms.Compose([transforms.Resize((150, 150)), transforms.ToTensor()])
caltech101 = datasets.Caltech101('path_to_save_caltech101', download=True, transform=transform)

In [ ]:

# 提取SIFT特征
def extract_sift_features(image):
    sift = cv2.SIFT_create()
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    kp, des = sift.detectAndCompute(gray, None)
    return des if des is not None else []

descriptors = [extract_sift_features(img) for img, _ in caltech101]
all_descriptors = np.vstack([d for d in descriptors if len(d) > 0])

In [ ]:

# 使用K-means创建视觉词袋
k = 2048
kmeans = MiniBatchKMeans(n_clusters=k, random_state=0).fit(all_descriptors)

# 为每个图像构建直方图
def build_histogram(descriptor, cluster_alg):
    histogram = np.zeros(len(cluster_alg.cluster_centers_))
    for d in descriptor:
        idx = cluster_alg.predict([d])
        histogram[idx] += 1
    return histogram

image_histograms = [build_histogram(d, kmeans) for d in descriptors]

In [ ]:

# 划分数据集
labels = [label for _, label in caltech101]
X_train, X_test, y_train, y_test = train_test_split(image_histograms, labels, test_size=0.2)

# 训练SVM
svms = []
for i in tqdm(range(101), desc="Training SVMs"):  # 假设有101个类别
    y_train_binary = (np.array(y_train) == i).astype(int)
    y_test_binary = (np.array(y_test) == i).astype(int)

    clf = svm.SVC(kernel='linear')
    clf.fit(X_train, y_train_binary)

    y_pred = clf.predict(X_test)
    print(f'Class {i} accuracy: {accuracy_score(y_test_binary, y_pred)}')

    svms.append(clf)